In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import datawig as dw
import mxnet as mx

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.datasets import make_moons
from sklearn.feature_selection import mutual_info_classif


from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier


%matplotlib inline

print("OK\n")

OK



In [2]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2934035270769239385
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9374597120
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7959507444548919135
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:22:00.0, compute capability: 7.5"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 1720110286
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11142282436875246064
 physical_device_desc: "device: 1, name: NVIDIA RTX A2000 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 2144165316]

In [4]:
app_train_origin =  pd.read_csv('./home-credit-default-risk/exports/app_train_res.csv')
app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test_res.csv')
app_train_rev_cash = pd.read_csv('./home-credit-default-risk/datasets/app_train_hc_rev-cash_axis-24-monthes.csv')
app_test_rev_cash = pd.read_csv('./home-credit-default-risk/datasets/app_test_hc_rev-cash_axis-24-monthes.csv')

In [5]:
# applicationのcolumnを取得する
app_columns = list(app_test_origin.select_dtypes(include='number').columns)
app_columns.remove('SK_ID_CURR')

In [ ]:
# 欠損値補完用のcolumnを取得
app_train_columns = list(app_test_origin.select_dtypes(include='number').columns)
app_test_columns = list(app_test_origin.select_dtypes(include='number').columns)

In [6]:
# applicationのオブジェクトをcopy
app_train = app_train_origin.copy(app_columns)
app_test = app_test_origin.copy(app_columns)

In [7]:
# IDを大文字へキャストする
app_train_rev_cash = app_train_rev_cash.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_rev_cash = app_test_rev_cash.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

In [8]:
# 参照対象の列名をリスト化
input_columns = app_columns.copy()
input_columns

['CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXP

In [9]:
# 処理対象の列名をリスト化(Rev/Cashのcolumnを取得する)
output_column = list(app_test_rev_cash.select_dtypes(include='number').columns)
output_column.remove('SK_ID_CURR')
output_column

['hc_re_cash_max_amt_annuity',
 'hc_re_cash_min_amt_annuity',
 'hc_re_cash_sum_amt_annuity',
 'hc_re_cash_avg_amt_annuity',
 'hc_re_cash_cnt_amt_annuity',
 'hc_re_cash_stddev_amt_annuity',
 'hc_re_cash_stddev_pop_amt_annuity',
 'hc_re_cash_var_pop_amt_annuity',
 'hc_re_cash_var_samp_amt_annuity',
 'hc_re_cash_variance_samp_amt_annuity',
 'hc_re_cash_max_amt_application',
 'hc_re_cash_min_amt_application',
 'hc_re_cash_sum_amt_application',
 'hc_re_cash_avg_amt_application',
 'hc_re_cash_cnt_amt_application',
 'hc_re_cash_stddev_amt_application',
 'hc_re_cash_stddev_pop_amt_application',
 'hc_re_cash_var_pop_amt_application',
 'hc_re_cash_var_samp_amt_application',
 'hc_re_cash_variance_samp_amt_application',
 'hc_re_cash_max_amt_credit',
 'hc_re_cash_min_amt_credit',
 'hc_re_cash_sum_amt_credit',
 'hc_re_cash_avg_amt_credit',
 'hc_re_cash_cnt_amt_credit',
 'hc_re_cash_stddev_amt_credit',
 'hc_re_cash_stddev_pop_amt_credit',
 'hc_re_cash_var_pop_amt_credit',
 'hc_re_cash_var_samp_amt_cr

In [10]:
# trainへマージ
app_train = pd.merge(app_train, app_train_rev_cash, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, app_test_rev_cash, on='SK_ID_CURR', how='left')

In [11]:
imputer = dw.SimpleImputer(
    input_columns=input_columns, # List: どのカラムで代入モデルを作成するか
    output_column=output_column[0], # どのカラムの欠損補完を行うか
    output_path='imputer_model', # 代入モデルの出力先
    num_hash_buckets=int(2 ** 15),
    num_labels=100,
    tokens='chars',
    numeric_latent_dim=100,
    numeric_hidden_layers=1,
    is_explainable=False
)
print("OK\n")

OK



In [12]:
imputer.fit(train_df=app_train,
            test_df=app_test,
            ctx=[mx.gpu()],
            learning_rate=4e-3,
            num_epochs=100,
            patience=3,
            test_split=0.1,
            weight_decay=0
)

RuntimeError: simple_bind error. Arguments:
numerical_features-2yW4Acq9GF: (16, 244)
hc_re_cash_max_amt_annuity: (16, 1)
Traceback (most recent call last):
  File "C:\Jenkins\workspace\mxnet-tag\mxnet\src\storage\storage.cc", line 119
MXNetError: Compile with USE_CUDA=1 to enable GPU usage